In [10]:
!pip install selenium
from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.common import NoSuchElementException, StaleElementReferenceException
from matplotlib import style
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import re

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable


In [32]:
driver = webdriver.Edge()
url = "https://zu.fang.com/house-a015277-b05/"
driver.get(url)

In [33]:
#def next_page(driver):
#    try:
#        next_page = WebDriverWait(driver, 10).until(
#            EC.presence_of_element_located((By.LINK_TEXT, "下一页"))
#        )
#        driver.execute_script("arguments[0].click();", next_page) 


In [34]:
i = 0
data = []
while i <= 2:
    houses = driver.find_elements(By.CSS_SELECTOR, 'dl.list.hiddenMap.rel')
    for house in houses:
        title = house.find_element(By.CSS_SELECTOR, 'p.title a').get_attribute('title')
        info = house.find_element(By.CSS_SELECTOR, 'p.font15.mt12.bold').text
        location = house.find_element(By.CSS_SELECTOR, 'p.gray6.mt12').text
        
        traffic_elements = house.find_elements(By.CSS_SELECTOR, 'span.note.subInfor')
        traffic = traffic_elements[0].text if traffic_elements else "无信息"
        
        price = house.find_element(By.CSS_SELECTOR, 'span.price').text
        
        item = {"名字": title, "信息": info, "地址": location, "交通": traffic, "价格": price}
        data.append(item)
    
    try:
        next_page = driver.find_element(By.LINK_TEXT, "下一页")
        next_page.click()
    except NoSuchElementException:
        print("NoSuchElementException")
    
    i += 1

NoSuchElementException
NoSuchElementException


In [35]:
houses_data = []

for item in data:
    title = item["名字"]
    info = item["信息"].split("|")
    price = item["价格"] 
    location = item["地址"]
    traffic = item["交通"]
    mode = info[0]
    layout = info[1]
    
    area = None
    if len(info) > 2 and "㎡" in info[2]:
        try:
            area = float(info[2].replace("㎡", "").strip())
        except ValueError:
            area = None
    
    orientation = info[3] if len(info) > 3 else "未知"
    
    house_dict = {
        '名字': title,
        '价格': price,
        '地址': location,
        '交通': traffic,
        '出租方式': mode,
        '布局': layout,
        '面积（㎡）': area,
        '朝向': orientation
    }

    houses_data.append(house_dict)  # 存入列表

df = pd.DataFrame(houses_data)

In [36]:
df.to_csv(r"C:\Users\18910\Desktop\北太平庄房租.csv",encoding='utf_8_sig')